In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from litQeval.eval_utils import *
import plotly.express as px
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import json

In [19]:
for i in json.load(open('data/queries.json')):
    if i["baseline"] == "Software Defect Prediction":
        baseline = i["baseline"]
        predicted = i["predicted"]
        break
data = get_data(baseline, predicted)
core_pubs = data["core_pubs"]
core_mean_embedding = data["core_mean_embedding"]
baseline_pubs = data["baseline_pubs"]
predicted_pubs = data["predicted_pubs"]
baseline_vs = data["baseline_vs"]
predicted_vs = data["predicted_vs"]
core_vs = data["core_vs"]
core_embeddigs = data["core_embeddings"]
core_threshold = data["core_threshold"]
core_embeddings = data["core_embeddings"]
predicted_embeddings = np.array([embedding for embedding in predicted_vs.get(include=["embeddings"])["embeddings"]])
baseline_embeddings = np.array([embedding for embedding in baseline_vs.get(include=["embeddings"])["embeddings"]])
core_mean_embedding.shape, predicted_embeddings.shape, baseline_embeddings.shape

Searching config file credentials for default 'live' instance..


Dimcli - Dimensions API Client (v1.3)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.10
Method: dsl.ini file


Starting iteration with limit=1000 skip=0 ...
0-1000 / 3795 (2.15s)
1000-2000 / 3795 (3.67s)
2000-3000 / 3795 (1.97s)
3000-3795 / 3795 (25.52s)
===
Records extracted: 3795


Total results: 3795


Searching config file credentials for default 'live' instance..


Total results after filtering: 3563
Dimcli - Dimensions API Client (v1.3)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.10
Method: dsl.ini file


Starting iteration with limit=1000 skip=0 ...
0-1000 / 14679 (9.59s)
1000-2000 / 14679 (6.20s)
2000-3000 / 14679 (3.49s)
3000-4000 / 14679 (2.97s)
4000-5000 / 14679 (2.83s)
5000-6000 / 14679 (2.95s)
6000-7000 / 14679 (2.76s)
7000-8000 / 14679 (26.38s)
8000-9000 / 14679 (3.35s)
9000-10000 / 14679 (5.52s)
10000-11000 / 14679 (5.94s)
11000-12000 / 14679 (2.69s)
12000-13000 / 14679 (6.18s)
13000-14000 / 14679 (3.22s)
14000-14679 / 14679 (2.92s)
===
Records extracted: 14679


Total results: 14679
Total results after filtering: 14651


100%|██████████| 3/3 [02:24<00:00, 48.26s/it]
c:\Users\saknini\Desktop\Master Project\LitQEval\.venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\saknini\Desktop\Master Project\LitQEval\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: Input contains NaN.

In [ ]:
cosine_sim = cosine_similarity(core_mean_embedding, baseline_embeddings).flatten()
baseline_pubs["similarity"] = cosine_sim

core_pubs_in_baseline = baseline_pubs[baseline_pubs["id"].isin(core_pubs)]
relevent_baseline_pubs = baseline_pubs[baseline_pubs["similarity"] >= core_threshold].copy()
print(f"Number of core publications in the baseline: {core_pubs_in_baseline.shape[0]}")
print(f"Number of relevant publications in the baseline: {relevent_baseline_pubs.shape[0]}")

In [ ]:
# predicted
cosine_sim = cosine_similarity(core_mean_embedding, predicted_embeddings).flatten()
predicted_pubs["similarity"] = cosine_sim

core_pubs_in_predicted = predicted_pubs[predicted_pubs["id"].isin(core_pubs)]
relevant_predicted_pubs = predicted_pubs[predicted_pubs["similarity"] >= core_threshold].copy()
print(f"Number of core publications in the predicted: {core_pubs_in_predicted.shape[0]}")
print(f"Number of relevant publications in the predicted: {relevant_predicted_pubs.shape[0]}")

### Cosine Similarity Measure

In [ ]:
recall = evaluate_recall(core_pubs, baseline_pubs, predicted_pubs)
# semnatic precision: every element that is more similar than the least similar core publication is considered relevant
# relevant_predicted_pubs: publications that are more similar than the least similar core publication.
pred_precision = relevant_predicted_pubs.shape[0] / predicted_pubs.shape[0] # total number of found publications
baseline_precision = (relevent_baseline_pubs.shape[0] / baseline_pubs.shape[0]) if baseline_pubs.shape[0] > 0 else 0
pred_f2 = fscore(pred_precision, recall["predicted_recall"], 2)
baseline_f2 = fscore(baseline_precision, recall["baseline_recall"], 2)
df = pd.DataFrame({
    "Semantic Precision": [pred_precision, baseline_precision],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic F2": [pred_f2, baseline_f2]
}, index=["Predicted", "Baseline"])
df

### Minimum Volume Enclosing Ellipsoid MMVE

In [ ]:
A, c = mvee(core_embeddings)
base_is_inside = is_inside_ellipse(A, c, baseline_embeddings)
predicted_is_inside = is_inside_ellipse(A, c, predicted_embeddings)

In [ ]:
mvve_prec_baseline = base_is_inside.sum() / len(base_is_inside)
mvve_prec_predicted = predicted_is_inside.sum() / len(predicted_is_inside)

mvve_df = pd.DataFrame({
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])
print(f"Baseline - Inside: {base_is_inside.sum()}")
print(f"Predicted - Inside: {predicted_is_inside.sum()}")
mvve_df

In [ ]:
results = pd.DataFrame({
    "Query": [predicted] + [baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic Precision": [pred_precision, baseline_precision],
    "Semantic F2": [pred_f2, baseline_f2],
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])

try:
    old_results = pd.read_excel("results.xlsx", index_col=0)
    results = pd.concat([old_results, results]).drop_duplicates(subset=["Query"]).round(3)
    results.to_excel("results.xlsx")
except FileNotFoundError:
    results.to_excel("results.xlsx")

display(results)

In [ ]:
results["Topic"] = results["Query"].copy()
results["Topic"][0::2] = np.nan
plt_df = results.bfill()
plt_df.reset_index(inplace=True)

In [ ]:
fig = px.histogram(plt_df, x="Topic", y=["Recall", "Semantic Precision", "MVVE Precision", "Semantic F2", "MVVE F2"], title="Metrics for Predicted and Baseline Queries", barmode="group",
                   facet_row="index", facet_row_spacing=0.1)

# Sort by value
fig.for_each_trace(lambda t: t.update(x=t.x[::-1], y=t.y[::-1]))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(yaxis1_title="", yaxis2_title="", yaxis1_dtick=0.2, yaxis2_dtick=0.2)
fig.show()

In [ ]:
# difference in metrics between predicted and baseline (Recall	Semantic Precision	Semantic F2	MVVE Precision	MVVE F2	)
diff_df = results.loc["Predicted"][["Recall","Semantic Precision" ,"Semantic F2" ,"MVVE Precision", "MVVE F2"]].values\
      - results.loc["Baseline"][["Recall","Semantic Precision" ,"Semantic F2" ,"MVVE Precision", "MVVE F2"]].values

diff_df = pd.DataFrame(diff_df.round(3), columns=["Recall","Semantic Precision" ,"Semantic F2" ,"MVVE Precision", "MVVE F2"], index=results.loc["Baseline"]["Query"])
diff_df = diff_df.style.map(lambda x: 'background-color: #6b0801' if x < -0.5 else 'background-color: #a82b22' if x < 0 else '')
diff_df.format("{:.3f}").set_caption("Difference in metrics between predicted and baseline (Red indicates a decrease in the metric in comparison to the baseline")